In [10]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

# Sample data - this would typically come from your RAG pipeline
data = {
    "question": ["What is the capital of France?", "Who wrote 'To Kill a Mockingbird'?"],
    "answer": ["Paris", "Harper Lee"],
    "contexts": [
        [
            "France is a country in Europe. Its capital is Paris.",
            "Paris is known as the city of love.",
        ],
        [
            "'To Kill a Mockingbird' was written by Harper Lee and published in 1960.",
            "Harper Lee was an American novelist.",
        ],
    ],
    "ground_truth": [
        "The capital of France is Paris.",
        "The author of 'To Kill a Mockingbird' is Harper Lee.",
    ],
}

# Convert to HuggingFace Dataset
dataset = Dataset.from_dict(data)

# Define the metrics you want to evaluate
metrics = [
    # faithfulness,       # Measures if the answer is supported by the context
    # answer_relevancy,   # Measures how relevant the answer is to the question
    context_recall,  # Measures if all relevant information is retrieved
    # context_precision   # Measures if the retrieved context is relevant
]

# Run the evaluation
result = evaluate(
    dataset=dataset,
    metrics=metrics,
)

# Print the results
print(result)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [11]:
from ragas.metrics.base import Metric


class CustomAnswerRelevancy(Metric):
    name = "custom_answer_relevancy"
    batch_size = 15

    def __init__(self):
        super().__init__()

    def score(self, row):
        # Implement your own relevancy scoring logic
        question = row["question"].lower()
        answer = row["answer"].lower()

        # Simple word overlap as example - replace with better logic
        q_words = set(question.split())
        a_words = set(answer.split())
        overlap = len(q_words & a_words) / len(q_words) if q_words else 0
        return overlap


# Usage
custom_metrics = [CustomAnswerRelevancy()]
result = evaluate(dataset, metrics=custom_metrics)

TypeError: Can't instantiate abstract class CustomAnswerRelevancy without an implementation for abstract method 'init'

In [28]:
import asyncio
from typing import List

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, answer_similarity, context_recall
from sentence_transformers import SentenceTransformer


class LocalEmbeddings:
    """Fully local embedding implementation"""

    def __init__(self, model_name="BAAI/bge-small-en-v1.5"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text, normalize_embeddings=True).tolist()

    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        return await asyncio.get_event_loop().run_in_executor(
            None, self.embed_documents, texts
        )

    async def aembed_query(self, text: str) -> List[float]:
        return await asyncio.get_event_loop().run_in_executor(
            None, self.embed_query, text
        )


# Initialize local components
local_embeddings = LocalEmbeddings()

# Select metrics that can run purely locally
metrics = [answer_relevancy, context_precision, context_recall, answer_similarity]

# Configure metrics to be fully local
for metric in metrics:
    # Remove any LLM dependencies
    if hasattr(metric, "llm"):
        metric.llm = None
    # Set local embeddings
    if hasattr(metric, "embedding"):
        metric.embedding = local_embeddings
    # Disable any OpenAI fallbacks
    if hasattr(metric, "api_key"):
        metric.api_key = ""

# Sample evaluation data
data = {
    "question": ["What is the capital of France?"],
    "answer": ["Paris"],
    "contexts": [["France is a country in Europe. Its capital is Paris."]],
    "ground_truth": ["The capital of France is Paris."],
}
dataset = Dataset.from_dict(data)

# Run fully local evaluation
result = evaluate(
    dataset=dataset,
    metrics=metrics,
)

# Print formatted results
print("\nRAGAS Evaluation Results (100% Local):")
for metric_name, score in result.items():
    print(f"{metric_name.replace('_', ' ').title():<20} {score:.4f}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [2]:
from typing import List

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_similarity, context_precision, context_recall


class LocalEmbeddings:
    """Fully local embedding implementation"""

    def __init__(self, model_name="BAAI/bge-small-en-v1.5"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text, normalize_embeddings=True).tolist()

    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        return await asyncio.get_event_loop().run_in_executor(
            None, self.embed_documents, texts
        )

    async def aembed_query(self, text: str) -> List[float]:
        return await asyncio.get_event_loop().run_in_executor(
            None, self.embed_query, text
        )


# Initialize local embedding model
local_embeddings = LocalEmbeddings()

# Patch the embedding model into the metrics
metrics = [context_precision, context_recall, answer_similarity]
for metric in metrics:
    if hasattr(metric, "embedding"):
        setattr(metric, "embedding", local_embeddings)
    if hasattr(metric, "llm"):
        setattr(metric, "llm", None)
    if hasattr(metric, "api_key"):
        setattr(metric, "api_key", "")

# Sample data for evaluation
data = {
    "question": ["What is the capital of France?"],
    "answer": ["Paris"],
    "contexts": [["France is a country in Europe. Its capital is Paris."]],
    "ground_truth": ["The capital of France is Paris."],
}
dataset = Dataset.from_dict(data)

# Evaluate
result = evaluate(dataset=dataset, metrics=metrics)

# Display results
print("\nRAGAS Evaluation Results (100% Local):")
for metric_name, score in result.items():
    print(f"{metric_name.replace('_', ' ').title():<20} {score:.4f}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
import os
import sys
import types

os.environ["OPENAI_API_KEY"] = ""

# Prevent openai from being loaded by any part of RAGAS
sys.modules["openai"] = types.ModuleType("openai")
sys.modules["openai"].OpenAI = None
sys.modules["openai"].api_key = ""

from typing import List

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_similarity, context_precision, context_recall


class LocalEmbeddings:
    def __init__(self, model_name="BAAI/bge-small-en-v1.5"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text, normalize_embeddings=True).tolist()

    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        return await asyncio.get_event_loop().run_in_executor(
            None, self.embed_documents, texts
        )

    async def aembed_query(self, text: str) -> List[float]:
        return await asyncio.get_event_loop().run_in_executor(
            None, self.embed_query, text
        )


local_embeddings = LocalEmbeddings()

metrics = [context_precision, context_recall, answer_similarity]
for metric in metrics:
    if hasattr(metric, "embedding"):
        metric.embedding = local_embeddings
    if hasattr(metric, "llm"):
        metric.llm = None

data = {
    "question": ["What is the capital of France?"],
    "answer": ["Paris"],
    "contexts": [["France is a country in Europe. Its capital is Paris."]],
    "ground_truth": ["The capital of France is Paris."],
}
dataset = Dataset.from_dict(data)

result = evaluate(dataset=dataset, metrics=metrics)

print("\nRAGAS Evaluation Results (100% Local):")
for metric_name, score in result.items():
    print(f"{metric_name.replace('_', ' ').title():<20} {score:.4f}")

ModuleNotFoundError: No module named 'openai.resources'; 'openai' is not a package